<a href="https://colab.research.google.com/github/GirishNautiyal/TSAI-END2-Session3/blob/main/END_Session3_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F


In [13]:
class DataGenerator(Dataset):
  def __init__(self, csv_file):
    self.data = pd.read_csv(csv_file)

  def __getitem__(self, index):
    r = self.data.iloc[index]
    label, image = r
    n = np.random.randint()
    sum = n + label
    return label, image, n, sum

  def __len__(self):
    return len(self.data)

In [3]:
# now let's work with FashionMnist

train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
train_loader = torch.utils.data.DataLoader(train_set
    ,batch_size=10
    ,shuffle=True
)

In [5]:
data_loader = torch.utils.data.DataLoader(
    train_set, 
    batch_size=10
)

batch = next(iter(data_loader))
len(batch)

2

In [6]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)

        return t

In [7]:
import torch.optim as optim

torch.set_grad_enabled(True)

In [8]:
images, labels = batch
network = Network()

In [9]:
preds = network(images)
loss = F.cross_entropy(preds, labels)
loss.item()

2.2925212383270264

In [10]:
optimizer = optim.Adam(network.parameters(), lr=0.01)

In [11]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [12]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

epoch 0 total_correct: 46673 loss: 347.91521322727203
epoch 1 total_correct: 51059 loss: 238.82222366333008
epoch 2 total_correct: 51832 loss: 218.73707754909992
epoch 3 total_correct: 52206 loss: 209.85032016038895
epoch 4 total_correct: 52330 loss: 205.1287147551775
epoch 5 total_correct: 52602 loss: 198.63719494640827
epoch 6 total_correct: 52744 loss: 196.3499694466591
epoch 7 total_correct: 52907 loss: 191.94737665355206
epoch 8 total_correct: 52878 loss: 189.4126254618168
epoch 9 total_correct: 53030 loss: 187.4637831747532


In [19]:
class Network2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t, t2):
        # (1) input layer
        t = t
        t2 = t2

        # (2) hidden conv layer
        t = self.conv1(t)
        t2 = self.conv1(t2)
        t = F.relu(t)
        t2 = F.relu(t2)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        t2 = F.max_pool2d(t2, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t2 = self.conv2(t2)
        t = F.relu(t)
        t2 = F.relu(t2)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        t2 = F.max_pool2d(t2, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t2 = t2.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t2 = self.fc1(t2)
        t = F.relu(t)
        t2 = F.relu(t2)

        # (5) hidden linear layer
        t = self.fc2(t)
        t2 = self.fc2(t2)
        t = F.relu(t)
        t2 = F.relu(t2)

        # (6) output layer
        t = self.out(t)
        t2 = self.out(t2)
        #t = F.softmax(t, dim=1)
        #t2 = F.softmax(t2, dim=1)

        return t, t2

In [36]:
images2, labels2 = batch
num = torch.tensor(np.random.randint(10))
print(num)
print(labels2)

tensor(3)
tensor([0, 1, 3, 5, 6, 0, 0, 9, 2, 9, 3, 0, 2, 8, 1, 1, 4, 2, 1, 8, 9, 4, 0, 8,
        0, 1, 3, 5, 3, 8, 4, 1, 9, 0, 0, 3, 1, 2, 1, 7, 7, 5, 8, 0, 2, 2, 1, 3,
        8, 8, 6, 7, 2, 2, 3, 5, 1, 2, 2, 4, 3, 6, 8, 4, 2, 0, 3, 8, 6, 6, 9, 3,
        5, 6, 0, 4, 4, 2, 9, 7, 3, 2, 4, 5, 4, 0, 4, 6, 6, 1, 4, 1, 7, 2, 8, 5,
        1, 3, 0, 5])


In [37]:
network2 = Network2()
preds2, summed = network2(images2, num)

RuntimeError: ignored

In [ ]:
loss = F.cross_entropy(preds2, labels)
loss.item()